# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sumg = 0
for sqrt_value in generator:
    sumg += sqrt_value

print(sumg)


8.382332347441762


In [ ]:
limit = 13
generator = square_root_generator(limit)
idx = 0
for sqrt_value in generator:
    print(idx, sqrt_value)
    idx += 1



0 1.0
1 1.4142135623730951
2 1.7320508075688772
3 2.0
4 2.23606797749979
5 2.449489742783178
6 2.6457513110645907
7 2.8284271247461903
8 3.0
9 3.1622776601683795
10 3.3166247903554
11 3.4641016151377544
12 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [1]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [2]:
import dlt
import duckdb

### 1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.

In [4]:
# define the connection to load to.
# We now use duckdb
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='workshop_dlt')

In [5]:
# we can load the next generator to the same or to a different table.
info = generators_pipeline.run(people_1(),
			table_name="people",
			write_disposition="replace")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 1.93 seconds
1 load package(s) were loaded to destination duckdb and into dataset workshop_dlt
The duckdb destination used duckdb:///d:\Workspace\ws_python\de_bootcamp\workshops\Workshop 1 - Data Ingestion\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707964781.8216665 is LOADED and contains no failed jobs


In [6]:
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [7]:
print("\n\n\n people table below:")

people = conn.sql("SELECT * FROM people").df()
display(people)




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707964781.8216665,idF9FVgfqAXQFA
1,2,Person_2,27,City_A,1707964781.8216665,z6QLGBOVSMdrmg
2,3,Person_3,28,City_A,1707964781.8216665,RaH2JdtvVnTzng
3,4,Person_4,29,City_A,1707964781.8216665,7Y3UUv7/c9IaIw
4,5,Person_5,30,City_A,1707964781.8216665,w2k7s58vnCP37g


In [9]:
# sum of ages of all people from first generator
f"Sum of Ages of first generator is {people.age.sum()}"

'Sum of Ages of first generator is 140'

### 2. Append the second generator to the same table as the first.

In [10]:
info = generators_pipeline.run(people_2(),
                            table_name="people",
							write_disposition="append")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.54 seconds
1 load package(s) were loaded to destination duckdb and into dataset workshop_dlt
The duckdb destination used duckdb:///d:\Workspace\ws_python\de_bootcamp\workshops\Workshop 1 - Data Ingestion\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707965048.7113085 is LOADED and contains no failed jobs


In [12]:
people = conn.sql("SELECT * FROM people").df()
display(people)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707964781.8216665,idF9FVgfqAXQFA,None
1,2,Person_2,27,City_A,1707964781.8216665,z6QLGBOVSMdrmg,None
2,3,Person_3,28,City_A,1707964781.8216665,RaH2JdtvVnTzng,None
3,4,Person_4,29,City_A,1707964781.8216665,7Y3UUv7/c9IaIw,None
4,5,Person_5,30,City_A,1707964781.8216665,w2k7s58vnCP37g,None
5,3,Person_3,33,City_B,1707965048.7113085,oajzApqxrbctNQ,Job_3
6,4,Person_4,34,City_B,1707965048.7113085,hwRdgiRTs3cBlg,Job_4
7,5,Person_5,35,City_B,1707965048.7113085,0omCwNUnYmr1uw,Job_5
8,6,Person_6,36,City_B,1707965048.7113085,2YVT8zweZ9w19A,Job_6
9,7,Person_7,37,City_B,1707965048.7113085,ZLpifjN6C7GAOQ,Job_7


### 3. **After correctly appending the data, calculate the sum of all ages of people.**

In [13]:
# sum of ages of all people from first generator
f"Sum of Ages of all generator is {people.age.sum()}"

'Sum of Ages of all generator is 353'

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [18]:
# we can load the next generator to the same or to a different table.
info = generators_pipeline.run(people_1(),
			table_name="people",
			write_disposition="replace")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.57 seconds
1 load package(s) were loaded to destination duckdb and into dataset workshop_dlt
The duckdb destination used duckdb:///d:\Workspace\ws_python\de_bootcamp\workshops\Workshop 1 - Data Ingestion\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707966118.3785625 is LOADED and contains no failed jobs


In [19]:
# we can load the next generator to the same or to a different table.
info = generators_pipeline.run(people_2(),
			table_name="people",
			write_disposition="merge",
            primary_key='id')

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.97 seconds
1 load package(s) were loaded to destination duckdb and into dataset workshop_dlt
The duckdb destination used duckdb:///d:\Workspace\ws_python\de_bootcamp\workshops\Workshop 1 - Data Ingestion\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707966148.03262 is LOADED and contains no failed jobs


In [16]:
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [20]:
tbl_people = conn.sql("SELECT * FROM people").df()
display(tbl_people)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707966118.3785625,yxoOrdSpS/HcRA,None
1,2,Person_2,27,City_A,1707966118.3785625,hoCA0Z1kHr1quA,None
2,7,Person_7,37,City_B,1707966148.03262,YN2i2xZgAOMG/Q,Job_7
3,4,Person_4,34,City_B,1707966148.03262,qPSJvWWRxqDS1w,Job_4
4,8,Person_8,38,City_B,1707966148.03262,lRMLnShppS6ZOA,Job_8
5,3,Person_3,33,City_B,1707966148.03262,8oaWAh7usa3Brw,Job_3
6,6,Person_6,36,City_B,1707966148.03262,4EnBa3bazeXEyw,Job_6
7,5,Person_5,35,City_B,1707966148.03262,QbOFmlgJ2/9sWw,Job_5


In [21]:
f"Sum of Ages of all generator is {tbl_people.age.sum()}"

'Sum of Ages of all generator is 266'

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX